In [ ]:
#%% Import Module
import torch
import sys
from IPython import get_ipython
sys.path.append('path_to_this_git')

from data.dataHandler import dataLoader
from API import TrainSegmentation, TestSegmentation, TrainClassification,TestClassification, CreateSubmission

Train with only BP-present images 

In [ ]:
ipy = get_ipython()
ipy.magic('matplotlib inline')
TrainSegmentation('./trainingConfig.json',
                  device = 'cuda:0',continueTrain=True,
                  dice_bg_weight=0.5)

Transfer learning with entire dataset including BP-absent Images

In [ ]:
ipy = get_ipython()
ipy.magic('matplotlib inline')
TrainSegmentation('./trainingConfig_BPabsent.json',device = 'cuda:0',
                  xferLearn=True,continueTrain=False,
                  dice_bg_weight=0.5)

Test Segmentation

In [ ]:
state_base_name = 'state_lr_0.001'
output_dir = 'output_moreBPabsent'#
epoch = 'best'
state_base_name = 'stateXfer_lr_0.0005'
TestSegmentation('./trainingConfig_BPabsent.json',
                  f'./{output_dir}/{state_base_name}_Epoch_{epoch}.pth.tar', 
                  f'./{output_dir}/result_test_data',
                  save_name= f'test_epoch_{epoch}_thresh',
                  device='cpu',
                  data_type=2, empty_thresh=5000)

Train Classification

In [ ]:
ipy = get_ipython()
ipy.magic('matplotlib inline')
TrainClassification('./trainingConfig_classification.json',device = 'cuda:0',
                    save_frequency=1,continueTrain=True)

Test Classification

In [ ]:
output_dir = 'output_classification(new)'
epoch ='best'
state_base_name = 'state_lr_0.001'
cutoff = 0
TestClassification('./trainingConfig_classification.json',
                  f'./{output_dir}/{state_base_name}_Epoch_{epoch}.pth.tar', 
                  f'./{output_dir}/result_test_data',
                  save_name= f'test_epoch_{epoch}_cutoff_{cutoff}',
                  device='cpu',
                  data_type=2, cutoff=cutoff)

Generate Submission

In [ ]:
ipy = get_ipython()
ipy.magic('matplotlib inline')
input_dir = './data/test'
output_dir = './submission'
submission_file_name = 'heyuan-5.csv'

cls_output_dir = 'output_classification'
cls_epoch ='best'
cls_state_base_name = 'state_lr_0.001'
cls_config_file = './trainingConfig_classification.json'
cls_net_state_dir = f'./{cls_output_dir}/{cls_state_base_name}_Epoch_{cls_epoch}.pth.tar'

seg_output_dir = 'output_moreBPabsent'
seg_epoch = 'best'
seg_state_base_name = 'stateXfer_lr_0.0005'
seg_config_file = './trainingConfig_BPabsent.json'
seg_net_state_dir = f'./{seg_output_dir}/{seg_state_base_name}_Epoch_{seg_epoch}.pth.tar'

CreateSubmission(input_dir, output_dir, submission_file_name, 
                 seg_config_file, seg_net_state_dir, 
                 cls_config_file, cls_net_state_dir,
                 seg_pixel_cutoff=5000,
                 cls_cutoff=-10,
                 device = 'cuda:0'
                 )